In [1]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb

In [2]:
wandb.init(entity="ameyar3103-iiit-hyderabad",project="recurrent_conv_art", config={
    "epochs": 5,
    "batch_size": 4,
    "learning_rate": 0.001,
    "model": "RecurrentCNN"
})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ameyar3103 (ameyar3103-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Data loading

In [3]:
df_train = pd.read_csv('wikiart_csv/artist_train.csv',header=None, names=["image_path", "artist_id"])
df_val = pd.read_csv('wikiart_csv/artist_val.csv',header=None, names=["image_path", "artist_id"])

In [4]:
# get the number of classes
num_classes = 23 # from artist_class.txt


In [5]:
# Gather input data
train_images = df_train['image_path'].values
train_labels = df_train['artist_id'].values

val_images = df_val['image_path'].values
val_labels = df_val['artist_id'].values

In [6]:
from torchvision import transforms
import cv2

## Preprocess data and create test and train dataset

In [7]:
# create test and train dataset for dataloader

def get_image(image_path,image_size=224):
    try:
        img = cv2.imread('./wikiart/' + image_path)
        if img is None:
            raise ValueError(f"Image not loaded: ./wikiart/{image_path}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape
        scale = 256 / min(h, w)
        new_w = int(w * scale)
        new_h = int(h * scale)
        img_resized = cv2.resize(img, (new_w, new_h))
        start_x = (new_w - image_size) // 2
        start_y = (new_h - image_size) // 2
        img_cropped = img_resized[start_y:start_y+image_size, start_x:start_x+image_size]
        img_cropped = img_cropped.astype(np.float32) / 255.0
        img_tensor = torch.from_numpy(img_cropped).permute(2, 0, 1)
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std  = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        img_tensor = (img_tensor - mean) / std
        return img_tensor
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return torch.zeros(3, image_size, image_size)

class WikiArtDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # image_vectors = []
        # for image in self.images:
        #     image_emb = get_image(image)
        #     image_vectors.append(image_emb)
        # image = torch.stack(image_vectors)
        image = self.images[idx]
        # label should be a one-hot encoded vector
        label = torch.zeros(num_classes)
        label[self.labels[idx]] = 1

        return image, label

train_dataset = WikiArtDataset(train_images, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = WikiArtDataset(val_images, val_labels)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

for i, (images, labels) in enumerate(train_loader):
    print(images)
    print(labels)
    break

('Impressionism/claude-monet_the-japanese-bridge-2-1924.jpg', 'Symbolism/nicholas-roerich_himalayas-1947-1.jpg', 'Impressionism/pierre-auguste-renoir_bather-drying-her-leg.jpg', 'Symbolism/salvador-dali_familia-ruth-moabitidis-1964.jpg', 'Romanticism/ivan-aivazovsky_the-bay-of-naples-by-moonlight-1892.jpg', 'Impressionism/pierre-auguste-renoir_umbrellas-1886.jpg', 'Impressionism/edgar-degas_two-harlequins-1886.jpg', 'Naive_Art_Primitivism/martiros-saryan_illustration-to-armenian-folk-tales-1937-9.jpg', 'Realism/pyotr-konchalovsky_the-roses-by-the-window-1953.jpg', 'Realism/ivan-shishkin_covert-1894.jpg', 'Impressionism/childe-hassam_lower-manhattan-aka-broad-and-wall-streets.jpg', 'Impressionism/pierre-auguste-renoir_christine-lerolle-embroidering-1897.jpg', 'Romanticism/gustave-dore_schismatics-mahomet.jpg', 'Realism/ilya-repin_moonlight-1896.jpg', 'Impressionism/edgar-degas_achille-de-gas-1872.jpg', 'Cubism/pablo-picasso_still-life.jpg', 'Realism/pyotr-konchalovsky_skating-rink-dynam

## Model architecture definition

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class RecurrentCNN(nn.Module):
    def __init__(self, num_classes, lstm_hidden_size=256, dropout_prob=0.5):
        super(RecurrentCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((14, 56))
        self.lstm_input_size = 64 * 56
        self.lstm_hidden_size = lstm_hidden_size
        self.lstm = nn.LSTM(input_size=self.lstm_input_size, hidden_size=lstm_hidden_size,
                            batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(2 * lstm_hidden_size, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))      
        x = self.pool1(x)              
        x = F.relu(self.conv2(x))    
        x = self.pool2(x)             
        x = self.adaptive_pool(x)      
        x = x.permute(0, 2, 1, 3).contiguous()  
        batch_size, seq_len, channels, width = x.shape
        x = x.view(batch_size, seq_len, channels * width)  
        lstm_out, _ = self.lstm(x)  
        x = lstm_out.mean(dim=1)    
        x = self.dropout(x)
        x = self.fc(x)              
        return x
    
model = RecurrentCNN(num_classes)
model.to('cuda')

import torch.optim as optim

wandb.watch(model, log="all")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Model training and saving the model,optimizer weights (to use when needed)

In [9]:
# Train the model
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for image_paths, labels in train_bar:
        image_tensors = torch.stack([get_image(image_path) for image_path in image_paths])
        images = image_tensors.to('cuda')
        labels = labels.to('cuda')
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_bar.set_postfix(loss=loss.item())
    
    avg_train_loss = running_loss / len(train_loader)
    wandb.log({"epoch": epoch+1, "train_loss": avg_train_loss})
    
    # Validation Loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        val_bar = tqdm(val_loader, desc="Validation")
        for image_paths, labels in val_bar:
            image_tensors = torch.stack([get_image(image_path) for image_path in image_paths])
            image_tensors = image_tensors.to('cuda')
            labels = labels.to('cuda')
            outputs = model(image_tensors)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.argmax(dim=1)).sum().item()
            val_bar.set_postfix(loss=loss.item())
    
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    wandb.log({"val_loss": avg_val_loss, "val_accuracy": val_accuracy})
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Validation: 100%|██████████| 90/90 [01:17<00:00,  1.17it/s, loss=3.11]


Epoch 1/5 - Train Loss: 2.5810, Val Loss: 2.4019, Val Accuracy: 28.90%


Validation: 100%|██████████| 90/90 [01:15<00:00,  1.19it/s, loss=2.41] 


Epoch 2/5 - Train Loss: 2.2534, Val Loss: 2.1157, Val Accuracy: 38.00%


Validation: 100%|██████████| 90/90 [01:14<00:00,  1.21it/s, loss=2.54] 


Epoch 3/5 - Train Loss: 2.0654, Val Loss: 2.0065, Val Accuracy: 40.24%


Validation: 100%|██████████| 90/90 [01:11<00:00,  1.25it/s, loss=1.92] 


Epoch 4/5 - Train Loss: 1.9179, Val Loss: 1.9156, Val Accuracy: 43.02%


Validation: 100%|██████████| 90/90 [01:11<00:00,  1.27it/s, loss=1.99] 

Epoch 5/5 - Train Loss: 1.7949, Val Loss: 1.8356, Val Accuracy: 45.50%


In [ ]:
num_epochs = 15

for epoch in range(5,5+num_epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for image_paths, labels in train_bar:
        image_tensors = torch.stack([get_image(image_path) for image_path in image_paths])
        images = image_tensors.to('cuda')
        labels = labels.to('cuda')
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_bar.set_postfix(loss=loss.item())
    
    avg_train_loss = running_loss / len(train_loader)
    wandb.log({"epoch": epoch+1, "train_loss": avg_train_loss})
    
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        val_bar = tqdm(val_loader, desc="Validation")
        for image_paths, labels in val_bar:
            image_tensors = torch.stack([get_image(image_path) for image_path in image_paths])
            image_tensors = image_tensors.to('cuda')
            labels = labels.to('cuda')
            outputs = model(image_tensors)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.argmax(dim=1)).sum().item()
            val_bar.set_postfix(loss=loss.item())
    
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    wandb.log({"val_loss": avg_val_loss, "val_accuracy": val_accuracy})
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    if(epoch%5==0):
        torch.save(model.state_dict(), f"recurrent_cnn_epoch_{epoch+1}.pth")
        torch.save(optimizer.state_dict(), f"recurrent_cnn_optimizer_epoch_{epoch+1}.pth")

Validation: 100%|██████████| 90/90 [01:22<00:00,  1.09it/s, loss=2.68] 


Epoch 6/15 - Train Loss: 1.6917, Val Loss: 1.7797, Val Accuracy: 47.65%


Validation: 100%|██████████| 90/90 [01:15<00:00,  1.19it/s, loss=1.89] 


Epoch 7/15 - Train Loss: 1.5727, Val Loss: 1.7408, Val Accuracy: 47.55%


Validation: 100%|██████████| 90/90 [01:17<00:00,  1.16it/s, loss=2.06] 


...

Epoch 18/15 - Train Loss: 0.1309, Val Loss: 2.2882, Val Accuracy: 50.81%


Validation: 100%|██████████| 90/90 [01:12<00:00,  1.24it/s, loss=2.08] 


Epoch 19/15 - Train Loss: 0.0909, Val Loss: 2.3468, Val Accuracy: 50.37%


Validation: 100%|██████████| 90/90 [01:10<00:00,  1.28it/s, loss=1.54] 


Epoch 20/15 - Train Loss: 0.0911, Val Loss: 2.2935, Val Accuracy: 50.74%
